In [19]:
train_data_dir = "data/train/"
test_data_dir = "data/test/"

train_data_dir = "/home/pierre/code/LRG-max/Skin_Cancer_Prevention/data/train/"
test_data_dir = "/home/pierre/code/LRG-max/Skin_Cancer_Prevention/data/test/"
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
# /home/pierre/code/LRG-max/Skin_Cancer_Prevention/data/train/malignant/2.jpg
img = load_img(f"{train_data_dir}malignant/2.jpg")
x = img_to_array(img)/255  # We divide the pixel value by 255 to end up between 0 and 1


img

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/malignant/2.jpg'